In [124]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


## INICIAR MODELO

In [149]:
model = pyo.ConcreteModel()

#Definir SET
model.INGREDIENTES = pyo.Set(initialize = ['Milho','Soja','Suplemento'])
model.RACAO = pyo.Set(initialize = ['A','B'])

#paramentro dicionarios
disponibilidade = {
    'Milho':10000,
    'Soja':8000,
    'Suplemento':2000
}

lucro={
    'A':10,
    'B':12
}
#nutriente = Proteina [0] , Energia [1]
nutrientes={
    'Milho':[15,8],
    'Soja':[20,5],
    'Suplemento':[30,3]
}
#Requisitos minimos Proteina[0], Energia[1]
requisitos={
    'A':[50,20],
    'B':[40,10],
}


#model. x    
# [i,j]           A            |   B
# milho       | model.x[0,0]   |
# soja        |                |
# suplemente  |                | 

# VARIAVEIS----------------------
model.x = pyo.Var(model.INGREDIENTES, model.RACAO, bounds=(0,2000),domain=pyo.NonNegativeIntegers)
#producao é a quantidade total
model.producao = pyo.Var(model.RACAO,bounds=(0,2000),domain=pyo.NonNegativeReals)

## Restrições
def proteina_total(model,r):

    proteina = sum(model.x[i,r] *  nutrientes[i][0] for i in model.INGREDIENTES)

    return  proteina >= requisitos[r][0] * model.producao[r]

model.proteina = pyo.Constraint(model.RACAO, rule=proteina_total)

def energia_total(model,r):

    energia = sum(model.x[i,r] * nutrientes[i][1] for i in model.INGREDIENTES)
    
    return energia >= requisitos[r][1] * model.producao[r]

model.energia = pyo.Constraint(model.RACAO, rule=energia_total)

def requisito_disponibilidade(model,i):

    return sum(model.x[i,r] for r in model.RACAO) <= disponibilidade[i]

model.restricao_disponibilidade = pyo.Constraint(model.INGREDIENTES, rule=requisito_disponibilidade)

#soma dos ingredientes é == producao
def restricao_producao(model,r):
  
    return sum(model.x[i, r] for i in model.INGREDIENTES ) == model.producao[r]

model.restricao_producao = pyo.Constraint(model.RACAO, rule=restricao_producao)

#objetivo

def objetivo1(model):

    #maximizar lucro, 
    #lucro é a produção de A e B * lucro de A e B
    return sum(
        model.producao[r] * lucro[r] for r in model.RACAO
    )
model.objetivo = pyo.Objective(rule=objetivo1, sense=pyo.maximize)


In [152]:
opt = SolverFactory('cplex', executable='C:\\CPLEX_Studio2211\\cplex\\bin\\x64_win64\\cplex.exe')
opt.options['TimeLimit'] = 180 
results = opt.solve(model, tee=True, warmstart=True)
print("-------FIM----------")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\DECIV\AppData\Local\Temp\tmpn8kni2vm.cplex.log' open.
CPLEX> New value for time limit in seconds: 180
CPLEX> Problem 'C:\Users\DECIV\AppData\Local\Temp\tmpsj31qxci.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> MIP start file 'C:\Users\DECIV\AppData\Local\Temp\tmpjm_bdqmd.cplex.mst' read.
CPLEX> Problem name         : C:\Users\DECIV\AppData\Local\Temp\tmpsj31qxci.pyomo.lp
Objective sense      : Maximize
Variables            :       8  [Box: 2,  General Integer: 6]
Objective nonzeros   :       2
Linear constraints   :       9  [Less: 7,  Equal: 2]
  Nonzeros           :     

In [153]:
for i in model.INGREDIENTES:
    for r in model.RACAO:
        print('----------')
        print('----------')
        print(f"Valor de x{i,r}: ",pyo.value(model.x[i,r]))
        print('------')        
        
        print(f'Producao total de {r}', pyo.value(model.producao[r]))



----------
----------
Valor de x('Milho', 'A'):  0.0
------
Producao total de A 0.0
----------
----------
Valor de x('Milho', 'B'):  0.0
------
Producao total de B 0.0
----------
----------
Valor de x('Soja', 'A'):  0.0
------
Producao total de A 0.0
----------
----------
Valor de x('Soja', 'B'):  0.0
------
Producao total de B 0.0
----------
----------
Valor de x('Suplemento', 'A'):  0.0
------
Producao total de A 0.0
----------
----------
Valor de x('Suplemento', 'B'):  0.0
------
Producao total de B 0.0


In [98]:
model.pprint()

2 Set Declarations
    INGREDIENTES : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Milho', 'Soja', 'Suplemento'}
    RACAO : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'A', 'B'}

2 Var Declarations
    producao : Size=2, Index=RACAO
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :   0.0 :  None : False : False : NonNegativeReals
          B :     0 :   0.0 :  None : False : False : NonNegativeReals
    x : Size=6, Index=INGREDIENTES*RACAO
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
             ('Milho', 'A') :     0 :   0.0 :  None : False : False : NonNegativeReals
             ('Milho', 'B') :     0 :   0.0 :  None : False : False : NonNegativeReals
              ('Soja', 'A') :     0 :   0.0 :  None : False : False : NonNegativeReals
              ('Soja',